Pseudo code

---- PREPARE DATA BY SPLITTING INTO INCREMENTAL LOAD

1. Convert to date data type
if yy > 25 then:
    Custom function to convert 20th century years dates to 1997 instead of 2097 using strftime
else:
    to_date

2. Function to iteratively for count distinct yyyy-mm months:
- dt = every date
- append 1 day's data to df
- save as '/date/csv_name' + dt + '.csv'
- dateadd a day

---- CONVERT EACH SAVED CSV FILE TO PARQUET AND SAVE IT TO 'LANDING' FOLDER

---- BRONZE LAYER: ConformInterface
1. Read parquet file from LANDING for a specific date and a schema.json
2. Write a pyspark code to verify parquet schema vs mentioned schema in json.
3. Move parquet file to BRONZE if verification is successful else throw error and stop processing.

---- SILVER LAYER: StandardizeColumns
1. Read parquet file from BRONZE for same specific date
2. Add audit columns to spark dataframe
3. Save dataframe to SILVER with same name and delete file in BRONZE

---- GOLD LAYER: ChangeDataCapture
1. Perform incremental load of only changed data

---- MODEL SQL DATABASE SCHEMA, IT WILL ACT AS WAREHOUSE (EXPLORE OPTIONS FOR FREE TIER)    -dbdiagram.io, erdlab.io

---- COPY GOLD DATA INTO SQL WAREHOUSE

---- USE DBT FREE TIER TO INTEGRATE, ANY TRANSFORMATION, CREATE EXTRACT TABLES

---- SERVE EXTRACT TABLE TO SERVING FOLDER AS CSV FOR ANALYSIS

---- CREATE POWERBI DASHBOARD

---- CREATE ML MODEL TO PREDICT FUTURE PRICES BASED ON DOMESTIC FACTORS SUCH AS MSP, PRODUCTION





In [35]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [4]:
spark = SparkSession.builder.getOrCreate()

In [11]:
rice_path = 'datasets/daily_retail_price_Rice_upto-apr_2015.csv'
df = spark.read.csv(rice_path, header=True)

In [26]:
df.show(5)

In [38]:
df1 = df.groupBy('Centre_Name').agg(countDistinct('Date')).alias('no_of_dates')
df1.toPandas().to_csv('locations.csv')

+--------+-----------+--------------+-----+
|    Date|Centre_Name|Commodity_Name|Price|
+--------+-----------+--------------+-----+
|25-11-97|      DELHI|          Rice|   10|
|25-11-97|     SHIMLA|          Rice|   12|
|25-11-97|    LUCKNOW|          Rice|  6.5|
|25-11-97|  AHMEDABAD|          Rice|   10|
|25-11-97|     BHOPAL|          Rice|    9|
+--------+-----------+--------------+-----+
only showing top 5 rows



In [52]:
df = df.withColumn("date_column", to_date(df["Date"], "dd-MM-yy"))

252.47058823529412